In [ ]:
# !pip install sentence-transformers faiss-cpu gpt4all llama-cpp-python scipy tqdm hf_xet

In [64]:
import os, json, sqlite3, textwrap
from datetime import datetime
from typing import List, Dict
import numpy as np

from sentence_transformers import SentenceTransformer
import faiss

In [65]:
SQLITE_DB = "crop_diseases_rag.db"
FAISS_INDEX_PATH = "crop_diseases.faiss"
FAISS_META_PATH = "faiss_metadata.json"
EMBED_MODEL_NAME = "all-MiniLM-L6-v2"   # small & fast
EMBED_DIM = 384
TOP_K = 3

In [66]:
crop_diseases_data = [
    # Corn - Common Rust
    {
        "label": "Corn___Common_Rust",
        "crop": "Corn",
        "disease": "Common Rust",
        "why_en": "Caused by the fungus Puccinia sorghi; develops in moderate temperatures (16–25°C) and high humidity; spreads by wind-borne spores.",
        "precautions_en": "Grow rust-resistant hybrids; rotate with non-host crops; destroy volunteer corn and grassy weeds.",
        "remedies_en": "Apply foliar fungicides (strobilurins or triazoles) when conditions favor disease; monitor regularly.",
        "why_hi": "यह रोग Puccinia sorghi कवक के कारण होता है; 16–25°C तापमान और उच्च आर्द्रता में बढ़ता है; हवा से फैलने वाले स्पोर्स के द्वारा फैलता है।",
        "precautions_hi": "रस्ट-प्रतिरोधी किस्में उगाएँ; नॉन-होस्ट फसलों के साथ रोटेशन करें; खेत में जंगली मक्का और घास नष्ट करें।",
        "remedies_hi": "जब बीमारी के अनुकूल मौसम हो तो पत्ती-छिड़काव के लिए स्ट्रोबिल्यूरिन या ट्रायाज़ोल समूह के फंगीसाइड का उपयोग करें; नियमित निगरानी रखें।"
    },
    # Corn - Gray Leaf Spot
    {
        "label": "Corn___Gray_Leaf_Spot",
        "crop": "Corn",
        "disease": "Gray Leaf Spot",
        "why_en": "Caused by Cercospora zeae-maydis; thrives in warm (25–30°C), humid conditions with prolonged leaf wetness; survives on residue.",
        "precautions_en": "Plant tolerant hybrids; rotate out of corn for 2+ years; bury or remove residue; avoid excessive plant density.",
        "remedies_en": "Apply fungicides at VT–R1 if disease develops and weather favors spread; improve airflow.",
        "why_hi": "यह Cercospora zeae-maydis कवक के कारण होता है; 25–30°C तापमान और नमी में बढ़ता है; संक्रमित अवशेष पर जीवित रहता है।",
        "precautions_hi": "सहनशील हाईब्रिड उगाएँ; 2+ वर्ष तक मक्का से अन्य फसल रखें; अवशेष को दफन या हटाएँ; अत्यधिक पौध घनत्व से बचें।",
        "remedies_hi": "यदि बीमारी बढ़ रही है और मौसम अनुकूल है तो VT–R1 अवधि में फंगीसाइड का छिड़काव करें; वायु प्रवाह बेहतर करें।"
    },
    # Corn - Healthy
    {
        "label": "Corn___Healthy",
        "crop": "Corn",
        "disease": "Healthy",
        "why_en": "No active disease infection detected.",
        "precautions_en": "Maintain crop rotation, sanitation, balanced fertilization, and monitor regularly.",
        "remedies_en": "Continue good agronomic practices and regular scouting.",
        "why_hi": "कोई सक्रिय रोग संक्रमण नहीं पाया गया है।",
        "precautions_hi": "फसल रोटेशन, खेत की सफाई, संतुलित उर्वरक प्रबंधन और नियमित सर्वेक्षण बनाए रखें।",
        "remedies_hi": "अच्छी कृषि तकनीकें और नियमित निरीक्षण जारी रखें।"
    },
    # Corn - Northern Leaf Blight
    {
        "label": "Corn___Northern_Leaf_Blight",
        "crop": "Corn",
        "disease": "Northern Leaf Blight",
        "why_en": "Caused by Exserohilum turcicum; develops in cool (18–27°C), humid weather; overwinters in crop debris.",
        "precautions_en": "Use resistant hybrids; rotate with non-host crops; till under infected residue; manage irrigation to reduce leaf wetness.",
        "remedies_en": "Apply fungicides early when lesions appear, especially at tasseling in high-risk fields.",
        "why_hi": "यह Exserohilum turcicum कवक के कारण होता है; 18–27°C के ठंडे और नम मौसम में बढ़ता है; फसल अवशेष पर बचा रहता है।",
        "precautions_hi": "प्रतिरोधी हाईब्रिडों का उपयोग करें; नॉन-होस्ट फसलों के साथ रोटेशन करें; संक्रमित अवशेष दफ़न करें; सिंचाई प्रबंधन करें।",
        "remedies_hi": "खुले घाव (lesions) दिखाई देने पर जल्दी फंगीसाइड छिड़काव करें, विशेषकर टैसलिंग के समय जोखिम वाले खेतों में।"
    },

    # Potato - Early Blight
    {
        "label": "Potato___Early_Blight",
        "crop": "Potato",
        "disease": "Early Blight",
        "why_en": "Caused by Alternaria solani; develops in warm (24–29°C), humid weather; often affects older leaves first.",
        "precautions_en": "Use certified seed tubers; rotate with non-solanaceous crops; maintain proper spacing and nutrition.",
        "remedies_en": "Apply protectant fungicides (chlorothalonil, mancozeb) when symptoms start; remove infected foliage.",
        "why_hi": "यह Alternaria solani कवक के कारण होता है; 24–29°C और नमी में बढ़ता है; आमतौर पर पुराने पत्तों को पहले प्रभावित करता है।",
        "precautions_hi": "प्रमाणित बीज आलू का उपयोग करें; 2+ साल के लिए गैर-सोलानासियस फसलों के साथ रोटेशन करें; उचित दूरी और पोषण बनाए रखें।",
        "remedies_hi": "लक्षण दिखने पर प्रोटेक्टेंट फंगीसाइड (जैसे क्लोरोथालोनिल, मैनकोज़ेब) का छिड़काव करें; संक्रमित पत्तियाँ हटाएँ।"
    },
    # Potato - Healthy
    {
        "label": "Potato___Healthy",
        "crop": "Potato",
        "disease": "Healthy",
        "why_en": "No active disease infection detected.",
        "precautions_en": "Plant certified, disease-free seed; maintain irrigation and scout regularly.",
        "remedies_en": "Continue integrated crop management to prevent outbreaks.",
        "why_hi": "कोई सक्रिय रोग संक्रमण नहीं पाया गया है।",
        "precautions_hi": "प्रमाणित रोग-रहित बीज उगाएँ; सिंचाई बनाए रखें और नियमित निरीक्षण करें।",
        "remedies_hi": "रोग-रोधी प्रबंधन जारी रखें।"
    },
    # Potato - Late Blight
    {
        "label": "Potato___Late_Blight",
        "crop": "Potato",
        "disease": "Late Blight",
        "why_en": "Caused by Phytophthora infestans; favoured by cool (15–20°C), wet conditions with prolonged leaf wetness; spreads rapidly.",
        "precautions_en": "Plant resistant varieties; avoid overhead irrigation; remove and destroy cull piles; use certified seed.",
        "remedies_en": "Apply systemic + protectant fungicides in a spray program; remove infected plants and improve drainage.",
        "why_hi": "यह Phytophthora infestans से होता है; 15–20°C और गीले वातावरण में, लंबे समय तक पत्ती गीली रहने पर फैलता है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; ऊपर से सिंचाई से बचें; कचरा / कूल पाइल्स नष्ट करें; प्रमाणित बीज प्रयोग करें।",
        "remedies_hi": "सिस्टमिक और प्रोटेक्टेंट फंगीसाइड के साथ छिड़काव करें; संक्रमित पौधों को हटाएँ और जल निकासी सुधारें।"
    },

    # Rice - Brown Spot
    {
        "label": "Rice___Brown_Spot",
        "crop": "Rice",
        "disease": "Brown Spot",
        "why_en": "Caused by Bipolaris oryzae; associated with drought stress, nutrient deficiencies (esp. K), and high humidity.",
        "precautions_en": "Use clean seed; maintain balanced fertilization (K especially); manage water to avoid stress periods.",
        "remedies_en": "Apply foliar fungicides if severe; correct nutrient deficiency and maintain proper water management.",
        "why_hi": "यह Bipolaris oryzae के कारण होता है; सूखा तनाव, पोषक तत्व की कमी (विशेषकर पोटेशियम) और उच्च आर्द्रता से जुड़ा है।",
        "precautions_hi": "साफ बीज का उपयोग करें; संतुलित उर्वरक (विशेषकर K) दें; पानी प्रबंधन ठीक रखें।",
        "remedies_hi": "यदि गंभीर हो तो पत्ती-छिड़काव फंगीसाइड का उपयोग करें; पोषक तत्व की कमी को सही करें।"
    },
    # Rice - Healthy
    {
        "label": "Rice___Healthy",
        "crop": "Rice",
        "disease": "Healthy",
        "why_en": "No active disease infection detected.",
        "precautions_en": "Maintain water depth, timely fertilizer application and weed control.",
        "remedies_en": "Continue integrated crop management.",
        "why_hi": "कोई सक्रिय रोग संक्रमण नहीं पाया गया है।",
        "precautions_hi": "सही जल स्तर, समय पर उर्वरक और घास नियंत्रण रखें।",
        "remedies_hi": "समेकित फसल प्रबंधन जारी रखें।"
    },
    # Rice - Leaf Blast
    {
        "label": "Rice___Leaf_Blast",
        "crop": "Rice",
        "disease": "Leaf Blast",
        "why_en": "Caused by Magnaporthe oryzae; favoured by high humidity, frequent rains, and excessive nitrogen.",
        "precautions_en": "Use blast-resistant varieties; avoid excessive late N; maintain good drainage and spacing.",
        "remedies_en": "Apply registered fungicides (e.g., tricyclazole where allowed) at early stages; manage water to reduce leaf wetness.",
        "why_hi": "यह Magnaporthe oryzae के कारण होता है; उच्च आर्द्रता और अधिक नाइट्रोजन अनुप्रयोग से बढ़ता है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; देर से अधिक N न दें; जल निकासी और दूरी बनाए रखें।",
        "remedies_hi": "लक्षण दिखने पर अधिकृत फंगीसाइड का छिड़काव करें; पत्ती गीली होने को कम करें।"
    },
    # Rice - Neck Blast
    {
        "label": "Rice___Neck_Blast",
        "crop": "Rice",
        "disease": "Neck Blast",
        "why_en": "Severe panicle infection by Magnaporthe; occurs under humid, cloudy weather at heading stage and can cause sterility/ yield loss.",
        "precautions_en": "Grow resistant varieties; avoid high late nitrogen; monitor fields at heading.",
        "remedies_en": "Spray recommended fungicides at booting/heading as per local recommendations; improve field conditions.",
        "why_hi": "यह अनाज के गले (panicle neck) पर Magnaporthe द्वारा गंभीर संक्रमण है; शुष्क मौसम में हानिकारक नहीं पर नम मौसम में नुकसान पहुंचाता है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; देर से अधिक N न दें; हेडिंग पर निगरानी रखें।",
        "remedies_hi": "बूटिंग/हैडिंग पर स्थानीय सलाह अनुसार फंगीसाइड का छिड़काव करें; खेत की स्थितियाँ सुधारें।"
    },

    # Sugarcane - Bacterial Blight
    {
        "label": "Sugarcane_Bacterial Blight",
        "crop": "Sugarcane",
        "disease": "Bacterial Blight",
        "why_en": "Caused by Xanthomonas spp.; spreads via infected cuttings, contaminated tools and rain splash.",
        "precautions_en": "Use disease-free seed cane; sanitize tools; avoid wounding during handling; control insect vectors.",
        "remedies_en": "Hot-water treatment of setts where recommended; remove and burn infected stools; maintain drainage.",
        "why_hi": "यह Xanthomonas बैक्टीरिया के कारण होता है; संक्रमित कटिंग्स, गंदे औज़ार और बारिश की बूदों से फैलता है।",
        "precautions_hi": "रोग-रहित सेट्स का उपयोग करें; औज़ार को साफ रखें; हैंडलिंग में चोट से बचें।",
        "remedies_hi": "जहाँ उपयुक्त हो सेट्स का हॉट-वॉटर ट्रीटमेंट करें; संक्रमित हिस्सों को हटाएँ और जल निकासी ठीक रखें।"
    },
    # Sugarcane - Healthy
    {
        "label": "Sugarcane_Healthy",
        "crop": "Sugarcane",
        "disease": "Healthy",
        "why_en": "No active disease infection detected.",
        "precautions_en": "Plant certified setts; monitor for pests and disease; maintain nutrition.",
        "remedies_en": "Maintain field hygiene and recommended agronomy.",
        "why_hi": "कोई सक्रिय रोग संक्रमण नहीं पाया गया है।",
        "precautions_hi": "प्रमाणित सेट्स लगाएँ; कीट और रोगों के लिए निगरानी रखें; पोषण बनाए रखें।",
        "remedies_hi": "खेत की सफाई और अनुशंसित कृषि बनाए रखें।"
    },
    # Sugarcane - Red Rot
    {
        "label": "Sugarcane_Red_Rot",
        "crop": "Sugarcane",
        "disease": "Red Rot",
        "why_en": "Caused by Colletotrichum falcatum; risk increases with drought stress, poor drainage and infected planting material.",
        "precautions_en": "Plant resistant varieties; use healthy setts; avoid waterlogging and stress.",
        "remedies_en": "Rogue infected clumps; treat seed cane; improve drainage and rotate fields.",
        "why_hi": "यह Colletotrichum falcatum के कारण होता है; सूखा तनाव, खराब जल निकासी और संक्रमित सामग्री से जोखिम बढ़ता है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; स्वस्थ सेट्स का प्रयोग करें; पानी जमाव से बचें।",
        "remedies_hi": "संक्रमित हिस्सों को हटाएँ; सेट्स पर उपचार करें; जल निकासी बेहतर करें।"
    },
    # Wheat - Brown Rust
    {
        "label": "Wheat___Brown_Rust",
        "crop": "Wheat",
        "disease": "Brown Rust",
        "why_en": "Caused by Puccinia triticina; spreads by wind-borne urediniospores; prefers mild, moist conditions.",
        "precautions_en": "Grow resistant cultivars; remove volunteer wheat; rotate crops; monitor weather.",
        "remedies_en": "Apply fungicides (triazoles) at onset if economic thresholds are exceeded.",
        "why_hi": "यह Puccinia triticina कवक के कारण होता है; हवा से फैलने वाले स्पोर्स द्वारा फैलता है; नम मौसम पसंद करता है।",
        "precautions_hi": "प्रतिरोधी किस्में उगाएँ; स्वयंबीन गेहूँ निकालें; रोटेशन करें; मौसम की निगरानी रखें।",
        "remedies_hi": "आर्थिक सीमा-पार होने पर फंगीसाइड (ट्रायाजोल) छिड़काव करें।"
    },
    # Wheat - Healthy
    {
        "label": "Wheat___Healthy",
        "crop": "Wheat",
        "disease": "Healthy",
        "why_en": "No active disease infection detected.",
        "precautions_en": "Grow resistant cultivars; maintain balanced fertilization and pest control.",
        "remedies_en": "Follow recommended sowing dates and crop management.",
        "why_hi": "कोई सक्रिय रोग संक्रमण नहीं पाया गया है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; संतुलित उर्वरक और कीट नियंत्रण रखें।",
        "remedies_hi": "अनुशंसित बोवाई तिथियाँ और प्रबंधन का पालन करें।"
    },
    # Wheat - Yellow Rust
    {
        "label": "Wheat___Yellow_Rust",
        "crop": "Wheat",
        "disease": "Yellow Rust",
        "why_en": "Caused by Puccinia striiformis; thrives in cool (8–15°C), moist weather; spreads rapidly by wind.",
        "precautions_en": "Grow resistant varieties; monitor fields; remove volunteer wheat or alternate hosts.",
        "remedies_en": "Apply fungicides (triazoles) at first signs; adjust sowing dates to avoid peak infection windows.",
        "why_hi": "यह Puccinia striiformis के कारण होता है; 8–15°C के ठंडे और नम मौसम में बढ़ता है; हवा से तेजी से फैलता है।",
        "precautions_hi": "प्रतिरोधी किस्में लगाएँ; खेतों की नियमित निगरानी करें; स्वयंबीन गेहूँ हटाएँ।",
        "remedies_hi": "प्रथम संकेत पर फंगीसाइड (ट्रायाजोल) छिड़काव करें; संक्रमण पीक से बचने के लिए बोने की तारीख समायोजित करें।"
    }
]


In [67]:
def init_db(path=SQLITE_DB, recreate=False):
    if recreate and os.path.exists(path):
        os.remove(path)
    conn = sqlite3.connect(path)
    cur = conn.cursor()
    cur.execute('''
    CREATE TABLE IF NOT EXISTS disease_kb (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        label TEXT UNIQUE,
        crop TEXT,
        disease TEXT,
        why_en TEXT,
        precautions_en TEXT,
        remedies_en TEXT,
        why_hi TEXT,
        precautions_hi TEXT,
        remedies_hi TEXT,
        content_en TEXT,
        inserted_at TEXT
    )''')
    conn.commit()

    # insert records if table empty
    cur.execute("SELECT COUNT(*) FROM disease_kb")
    count = cur.fetchone()[0]
    if count == 0:
        for r in crop_diseases_data:
            content_en = f"{r['crop']} {r['disease']} - Why: {r['why_en']} Precautions: {r['precautions_en']} Remedies: {r['remedies_en']}"
            cur.execute('''INSERT OR IGNORE INTO disease_kb
                           (label, crop, disease, why_en, precautions_en, remedies_en, why_hi, precautions_hi, remedies_hi, content_en, inserted_at)
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                        (r['label'], r['crop'], r['disease'], r['why_en'], r['precautions_en'], r['remedies_en'],
                         r['why_hi'], r['precautions_hi'], r['remedies_hi'], content_en, datetime.utcnow().isoformat()))
    conn.commit()
    conn.close()

# initialize DB (set recreate=True first run if you want to overwrite)
init_db(recreate=True)
print("DB created/populated at", SQLITE_DB)

DB created/populated at crop_diseases_rag.db


In [68]:
embedder = SentenceTransformer(EMBED_MODEL_NAME)

def build_faiss(sqlite_path=SQLITE_DB, faiss_path=FAISS_INDEX_PATH, meta_path=FAISS_META_PATH, force_rebuild=False):
    # load docs
    conn = sqlite3.connect(sqlite_path)
    cur = conn.cursor()
    cur.execute("SELECT id, content_en FROM disease_kb ORDER BY id")
    rows = cur.fetchall()
    conn.close()

    ids = [r[0] for r in rows]
    texts = [r[1] for r in rows]

    if os.path.exists(faiss_path) and not force_rebuild:
        try:
            index = faiss.read_index(faiss_path)
            print("Loaded FAISS index from disk.")
            # load metadata
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
            return index, meta["sqlite_ids"], texts
        except Exception as e:
            print("Failed to load FAISS index, rebuilding:", e)

    # compute embeddings
    print("Computing embeddings for", len(texts), "documents...")
    embs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True).astype('float32')
    # normalize for cosine similarity
    faiss.normalize_L2(embs)
    index = faiss.IndexFlatIP(EMBED_DIM)
    index.add(embs)
    # persist
    faiss.write_index(index, faiss_path)
    meta = {"sqlite_ids": ids}
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("FAISS built and saved.")
    return index, ids, texts

index, sqlite_ids, docs = build_faiss()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Computing embeddings for 17 documents...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]

FAISS built and saved.


In [69]:
def retrieve_by_text(query: str, top_k:int = TOP_K) -> List[Dict]:
    q_emb = embedder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    results = []
    for score, idx in zip(D[0].tolist(), I[0].tolist()):
        if idx < 0 or idx >= len(sqlite_ids):
            continue
        sqlite_id = sqlite_ids[idx]
        conn = sqlite3.connect(SQLITE_DB)
        cur = conn.cursor()
        cur.execute("SELECT label, crop, disease, why_en, precautions_en, remedies_en, why_hi, precautions_hi, remedies_hi FROM disease_kb WHERE id=?", (sqlite_id,))
        row = cur.fetchone()
        conn.close()
        if not row:
            continue
        results.append({
            "label": row[0],
            "crop": row[1],
            "disease": row[2],
            "why_en": row[3],
            "precautions_en": row[4],
            "remedies_en": row[5],
            "why_hi": row[6],
            "precautions_hi": row[7],
            "remedies_hi": row[8],
            "score": float(score)
        })
    return results

def get_info_by_label(label:str, top_k:int=1) -> Dict:
    # exact lookup first
    conn = sqlite3.connect(SQLITE_DB)
    cur = conn.cursor()
    cur.execute("SELECT label, crop, disease, why_en, precautions_en, remedies_en, why_hi, precautions_hi, remedies_hi FROM disease_kb WHERE label=?", (label,))
    row = cur.fetchone()
    conn.close()
    if row:
        return {
            "label": row[0],
            "crop": row[1],
            "disease": row[2],
            "english": {
                "why": row[3],
                "precautions": row[4],
                "remedies": row[5]
            },
            "hindi": {
                "why": row[6],
                "precautions": row[7],
                "remedies": row[8]
            },
            "source": "local_db"
        }
    # fallback to embedding-based retrieval using normalized text
    fallback_query = label.replace("___"," ").replace("_"," ")
    retrieved = retrieve_by_text(fallback_query, top_k=top_k)
    if not retrieved:
        return {"error": "not found"}
    top = retrieved[0]
    return {
        "label": top["label"],
        "crop": top["crop"],
        "disease": top["disease"],
        "english": {"why": top["why_en"], "precautions": top["precautions_en"], "remedies": top["remedies_en"]},
        "hindi": {"why": top["why_hi"], "precautions": top["precautions_hi"], "remedies": top["remedies_hi"]},
        "score": top["score"],
        "source": "embedding_retrieval"
    }

def get_info_by_text(query:str, top_k:int=1) -> Dict:
    retrieved = retrieve_by_text(query, top_k=top_k)
    if not retrieved:
        return {"error": "not found"}
    top = retrieved[0]
    return {
        "label": top["label"],
        "crop": top["crop"],
        "disease": top["disease"],
        "english": {"why": top["why_en"], "precautions": top["precautions_en"], "remedies": top["remedies_en"]},
        "hindi": {"why": top["why_hi"], "precautions": top["precautions_hi"], "remedies": top["remedies_hi"]},
        "score": top["score"],
        "source": "embedding_retrieval"
    }

In [70]:
tests = [
    "Potato___Late_Blight",
    "Wheat___Yellow_Rust",
    "Corn Gray Leaf Spot",
    "Rice Leaf Blast",
    "Sugarcane Red Rot"
]
for t in tests:
    print("="*60)
    print("Query/Label:", t)
    res = get_info_by_label(t) if "___" in t else get_info_by_text(t)
    print("Source:", res.get("source"))
    print("Score (if present):", res.get("score"))
    print("English - WHY:", res["english"]["why"])
    print("Hindi - WHY:", res["hindi"]["why"])
    print()

Query/Label: Potato___Late_Blight
Source: local_db
Score (if present): None
English - WHY: Caused by Phytophthora infestans; favoured by cool (15–20°C), wet conditions with prolonged leaf wetness; spreads rapidly.
Hindi - WHY: यह Phytophthora infestans से होता है; 15–20°C और गीले वातावरण में, लंबे समय तक पत्ती गीली रहने पर फैलता है।

Query/Label: Wheat___Yellow_Rust
Source: local_db
Score (if present): None
English - WHY: Caused by Puccinia striiformis; thrives in cool (8–15°C), moist weather; spreads rapidly by wind.
Hindi - WHY: यह Puccinia striiformis के कारण होता है; 8–15°C के ठंडे और नम मौसम में बढ़ता है; हवा से तेजी से फैलता है।

Query/Label: Corn Gray Leaf Spot
Source: embedding_retrieval
Score (if present): 0.7962436676025391
English - WHY: Caused by Cercospora zeae-maydis; thrives in warm (25–30°C), humid conditions with prolonged leaf wetness; survives on residue.
Hindi - WHY: यह Cercospora zeae-maydis कवक के कारण होता है; 25–30°C तापमान और नमी में बढ़ता है; संक्रमित अवशेष पर